# [Selected Problems in Machine Learning (NPFL097)](https://ufal.mff.cuni.cz/courses/npfl097)

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from glob import glob